In [12]:
#import os
#import geopandas as gpd
#from geopandas import GeoSeries
#import pandas as pd
#from geocube.api.core import make_geocube
#import rasterio
#import numpy as np
#import sys

In [14]:
#calcutes the parcel centroids and the area. 
def calculate_geo(x, path):
    import os
    import geopandas as gpd
    from geopandas import GeoSeries
    import pandas as pd
    def getXY(pt):
        return (pt.x, pt.y)
    polygon = gpd.read_file(path+'\\'+x+"_parcel_v2.geojson")
    points = gpd.read_file(path+'\\'+x+"_points.geojson")
    #reprojecting the polygons
    polygon = polygon.to_crs(4269)
    Parcel_Centroid = GeoSeries(polygon['geometry']).centroid
    x,y = [list(t) for t in zip(*map(getXY, Parcel_Centroid))]
    polygon['Centroid_X'] = x
    polygon['Centroid_Y'] = y
    polygon = polygon.to_crs(5070)
    polygon["PARCEL_AREA"] = polygon['geometry'].area/ 10**6
    #DMP points data had duplicates that were propogating errors throughout the whole join process
    points = points.drop_duplicates(['PRCLDMPID'], keep='first')
    #there left table has to be a gdf and the right df HAS to be a simple df to result in a gdf
    state = polygon.merge(pd.DataFrame(points.drop(columns='geometry')), on='PRCLDMPID', how= 'left')
    return state